<a href="https://colab.research.google.com/github/AnujKrishnaPhuyal/DeepLearning-Practices/blob/main/pytorch_to_tflite_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/main

/content/drive/MyDrive/main


In [ ]:
!ls

2.jpg  anuj_achyut  mnist.onnx	mobilenet.onnx	my_model.pth  onnx-tensorflow


In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 64.4 MB/s eta 0:00:00


In [ ]:
import torch
import tensorflow as tf
import onnx
# Load the PyTorch model
map_location=torch.device('cpu')
model = torch.load("my_model.pth",map_location)
# Get the model's state dictionary (i.e., the parameters)
state_dict = model.state_dict()
print(model)

# Print the names and values of the parameters
# for name, param in state_dict.items():
#     print(name, param)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [ ]:
# # Export the model to ONNX format
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "mobile_net.onnx", input_names=["input"], output_names=["output"])

# # Load the ONNX model
onnx_model = onnx.load("mobile_net.onnx")


============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
!pip install onnx_tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 54.6 MB/s eta 0:00:00


In [ ]:
from onnx_tf.backend import prepare
import onnx

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
tf_rep = prepare(onnx_model)
tf_rep.export_graph('model_tf')


None


In [ ]:
saved_model_dir = '/model_tf/saved_model.pb'
tflite_model_path = 'model.tflite'

print(saved_model_dir)

/model_tf/saved_model.pb


In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

OSError: ignored

#Analyzing tflite model 

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model into memory
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get the input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print the input and output details
print("Input details:", input_details)
print("Output details:", output_details)

# Get the model weights and other parameters
model_parameters = interpreter.get_tensor_details()

# Print the names and shapes of the model parameters
for i, parameter in enumerate(model_parameters):
    print("Parameter %d: %s, shape: %s" % (i, parameter["name"], parameter["shape"]))

# Get the input tensor shape
input_shape = input_details[0]["shape"]

# Create a random input tensor
input_data = np.random.random_sample(input_shape).astype(np.float32)

# Set the input tensor data
interpreter.set_tensor(input_details[0]["index"], input_data)

# Run the inference
interpreter.invoke()

# Get the output tensor data
output_data = interpreter.get_tensor(output_details[0]["index"])

# Print the output tensor data
print("Output data:", output_data)


Input details: [{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  1,   3, 224, 224], dtype=int32), 'shape_signature': array([  1,   3, 224, 224], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'PartitionedCall:0', 'index': 397, 'shape': array([   1, 1000], dtype=int32), 'shape_signature': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Parameter 0: serving_default_input:0, shape: [  1   3 224 224]
Parameter 1: add_5;transpose_11;mul_2;mul/x;add_1/y;convolution_2;Const_100;Add_4;convolution_3, shape: [16]
Parameter 2: add_19;transpose_44;mul_7;mul/x;add_1/y;convol